In [2]:
import sys
import torch
import numpy as np
import scipy.signal
from torchvision.io import read_image
import torch.nn as nn
import torchvision 
from torchvision import transforms 
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

In [3]:
#一维卷积
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)    #使用 NumPy 的 zeros 函数创建了一个长度为 p 的一维数组，数组中的所有元素都是 0
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])   #使用 NumPy 的 concatenate 函数将三个数组拼接在一起
    res = []
    for i in range(0, (int((len(x_padded) - len(w_rot))/s) + 1) * s, s):
        res.append(np.sum(x_padded[i:i + w_rot.shape[0]] * w_rot))
    return np.array(res)


## Testing:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

In [4]:
print('Conv1d Implementation:',
      conv1d(x, w, p=2, s=1))

print('Numpy Results:',
      np.convolve(x, w, mode='same')) 

Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results: [ 5 14 16 26 24 34 19 22]


In [5]:
#二维卷积
def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1,::-1]#取反
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0],
    p[1]:p[1]+X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, (int((X_padded.shape[0] -
                            W_rot.shape[0]) / s[0]) + 1) * s[0], s[0]):
        res.append([])
        for j in range(0, (int((X_padded.shape[1] -
                                W_rot.shape[1]) / s[1]) + 1) * s[1], s[1]):
            X_sub = X_padded[i:i + W_rot.shape[0],
                    j:j + W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    return(np.array(res))

X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

In [6]:
print('Conv2d Implementation:\n',
    conv2d(X, W, p=(1, 1), s=(1, 1)))


print('SciPy Results:\n',
    scipy.signal.convolve2d(X, W, mode='same'))

Conv2d Implementation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]
SciPy Results:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


In [7]:
img = read_image('example-image.png') 
 
print('Image shape:', img.shape)
print('Number of channels:', img.shape[0])#通道数
print('Image data type:', img.dtype)
print(img[:, 100:102, 100:102])

Image shape: torch.Size([3, 252, 221])
Number of channels: 3
Image data type: torch.uint8
tensor([[[179, 182],
         [180, 182]],

        [[134, 136],
         [135, 137]],

        [[110, 112],
         [111, 113]]], dtype=torch.uint8)


In [8]:
###14.2.2使用L2范数和dropout对神经网络正则化
#此处将指定层的L2惩罚项添加到损失函数中
loss_func = nn.BCELoss()    #定义了一个二分类交叉熵损失函数BCELoss()
loss = loss_func(torch.tensor([0.9]), torch.tensor([1.0]))  #该损失值基于预测值0.9和真实标签1.0
l2_lambda = 0.001

conv_layer = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5)   
l2_penalty = l2_lambda * sum([(p**2).sum() for p in conv_layer.parameters()])    #对于卷积层，计算了所有参数的平方和，然后乘以l2_lambda得到L2正则化项。
loss_with_penalty = loss + l2_penalty

linear_layer = nn.Linear(10, 16)
l2_penalty = l2_lambda * sum([(p**2).sum() for p in linear_layer.parameters()])
loss_with_penalty = loss + l2_penalty



In [9]:
# ## Loss Functions for Classification分类任务的损失函数
#  * **`nn.BCELoss()`** 二分类
#    * `from_logits=False` 
#    * `from_logits=True`
#  * **`nn.CrossEntropyLoss()`**多分类
#    * `from_logits=False`
#    * `from_logits=True`
##使用PyTorch计算二分类交叉熵损失（BCE）和多分类交叉熵损失（CCE，也称为多分类交叉熵损失）
####### Binary Cross-entropy
logits = torch.tensor([0.8])
probas = torch.sigmoid(logits)#将logits通过sigmoid函数转换为概率。
target = torch.tensor([1.0])

bce_loss_fn = nn.BCELoss()#使用BCELoss计算概率和目标标签之间的损失。这里的目标标签应该是0到1之间的浮点数，表示真实类别为1的概率（在二分类问题中）。
bce_logits_loss_fn = nn.BCEWithLogitsLoss()#使用BCEWithLogitsLoss，它内部会先对logits应用sigmoid函数，然后再计算BCE。这样做的好处是数值上更稳定，并且计算效率更高。

print(f'BCE (w Probas): {bce_loss_fn(probas, target):.4f}') #输入值是使用logits计算的类别概率
print(f'BCE (w Logits): {bce_logits_loss_fn(logits, target):.4f}')  #输入值是logits值

 
####### Categorical Cross-entropy
logits = torch.tensor([[1.5, 0.8, 2.1]])
probas = torch.softmax(logits, dim=1)
target = torch.tensor([2])

cce_loss_fn = nn.NLLLoss()  
cce_logits_loss_fn = nn.CrossEntropyLoss()

print(f'CCE (w Logits): {cce_logits_loss_fn(logits, target):.4f}')  #根据logits值计算对数概率
print(f'CCE (w Probas): {cce_loss_fn(torch.log(probas), target):.4f}')

BCE (w Probas): 0.3711
BCE (w Logits): 0.3711
CCE (w Logits): 0.5996
CCE (w Probas): 0.5996
